In [ ]:
%reset -f
from datetime import timedelta, date, datetime
import time
import pandas as pd
import numpy as np
import ccxt
import requests
import traceback
import logging
import threading
from threading import Thread

class ohlcvs():

    def binance_margin(self,timeframe,limit):
        pairs = pd.read_pickle("pairs_binance_margin.pkl")
        print('number of pairs in binance_margin: ',len(pairs))
        save_pairs=pairs.copy()
        data_pairs=pairs.copy()
        msec = 1000
        hold = 1

        i=0
        
        while i<len(pairs):
            time.sleep(hold)
            timeframes = (['1m',1,'minutes',limit*1,60*msec],['3m',3,'minutes',limit*3,60*msec],['5m',5,'minutes',limit*5,60*msec],['15m',15,'minutes',limit*15,60*msec],['30m',30,'minutes',limit*30,60*msec],['1h',1,'hours',limit*1,60*60*msec],['2h',2,'hours',limit*2,60*60*msec],['4h',4,'hours',limit*4,60*60*msec],['6h',6,'hours',limit*6,60*60*msec],['8h',8,'hours',limit*8,60*60*msec],['12h',12,'hours',limit*12,60*60*msec],['1d',1,'days',limit*1,60*60*24*msec],['3d',3,'days',limit*3,60*60*24*msec],['1w',1,'weeks',limit*7,60*60*24*7*msec])
            timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit' , 'period'])
            timeframes.set_index('timeframe', inplace=True)

            timeframes['limit']  = timeframes['limit'].astype(np.float64)
            timeframes['period'] = timeframes['period'].astype(np.float64)
            exchange = ccxt.binance({'enableRateLimit': True})

            now = exchange.milliseconds()
            if timeframes.loc[timeframe, 'name_time'] == 'minutes':
                from_datetime=pd.to_datetime(now, unit='ms') - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

            if timeframes.loc[timeframe, 'name_time'] == 'hours':
                from_datetime=pd.to_datetime(now, unit='ms') - timedelta(hours=timeframes.loc[timeframe, 'limit'])

            if timeframes.loc[timeframe, 'name_time'] == 'days':
                from_datetime=pd.to_datetime(now, unit='ms') - timedelta(days=timeframes.loc[timeframe, 'limit'])

            from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S%z')
            from_timestamp = exchange.parse8601(from_datetime)
            data = []

            while from_timestamp < now:

                try:
                    
                    ohlcvs = exchange.fetch_ohlcv(save_pairs.at[i,'pair'], timeframe, from_timestamp)
                    print('binance_margin:',i)

                    if len(ohlcvs) > 0:
                        first = ohlcvs[0][0]
                        last = ohlcvs[-1][0]

                        from_timestamp = ohlcvs[-1][0] + timeframes.loc[timeframe, 'period'] * timeframes.loc[timeframe, 'time']
                        data += ohlcvs

                    df = pd.DataFrame(data, columns=['Timestamp','open','high','low','close', 'volume'])

                    df['Timestamp'] = pd.DataFrame(df['Timestamp'].apply(exchange.iso8601))

                    i+=1


                except (Exception,ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout, ccxt.BadSymbol) as error:
                    print(logging.error(traceback.format_exc()))
                    i+=1
                    print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
                    time.sleep(hold)
                else:
                    data_pairs.at[i-1,'pair']=df
        data_pairs.to_pickle("data_pairs_binance_margin_%s.pkl" % (timeframe))

        return data_pairs

    def binance_future(self,timeframe,limit):
        pairs = pd.read_pickle("pairs_binance_future.pkl")
        print('number of pairs in binance_future: ',len(pairs))


        save_pairs=pairs.copy()
        data_pairs=pairs.copy()
        counter=0
        msec = 1000
        hold = 1
        i=0
        
        while i<len(pairs):
            timeframes = (['1m',1,'minutes',limit*1,60*msec],['3m',3,'minutes',limit*3,60*msec],['5m',5,'minutes',limit*5,60*msec],['15m',15,'minutes',limit*15,60*msec],['30m',30,'minutes',limit*30,60*msec],['1h',1,'hours',limit*1,60*60*msec],['2h',2,'hours',limit*2,60*60*msec],['4h',4,'hours',limit*4,60*60*msec],['6h',6,'hours',limit*6,60*60*msec],['8h',8,'hours',limit*8,60*60*msec],['12h',12,'hours',limit*12,60*60*msec],['1d',1,'days',limit*1,60*60*24*msec],['3d',3,'days',limit*3,60*60*24*msec],['1w',1,'weeks',limit*7,60*60*24*7*msec])
            timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit' , 'period'])
            timeframes.set_index('timeframe', inplace=True)

            timeframes['limit']  = timeframes['limit'].astype(np.float64)
            timeframes['period'] = timeframes['period'].astype(np.float64)


            exchange = ccxt.binanceusdm({'enableRateLimit': True})
            now = exchange.milliseconds()
            if timeframes.loc[timeframe, 'name_time'] == 'minutes':
                from_datetime=pd.to_datetime(now, unit='ms') - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

            if timeframes.loc[timeframe, 'name_time'] == 'hours':
                from_datetime=pd.to_datetime(now, unit='ms') - timedelta(hours=timeframes.loc[timeframe, 'limit'])

            if timeframes.loc[timeframe, 'name_time'] == 'days':
                from_datetime=pd.to_datetime(now, unit='ms') - timedelta(days=timeframes.loc[timeframe, 'limit'])



            from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S%z')
            from_timestamp = exchange.parse8601(from_datetime)
            data = []

            while from_timestamp < now:
                try:
                    
                    ohlcvs = exchange.fetch_ohlcv(save_pairs.at[i,'pair'], timeframe, from_timestamp)
                    print('binance_future:',i)

                    if len(ohlcvs) > 0:
                        first = ohlcvs[0][0]
                        last = ohlcvs[-1][0]

                        from_timestamp = ohlcvs[-1][0] + timeframes.loc[timeframe, 'period'] * timeframes.loc[timeframe, 'time']
                        data += ohlcvs

                    df = pd.DataFrame(data, columns=['Timestamp','open','high','low','close', 'volume'])
                    df['Timestamp'] = pd.DataFrame(df['Timestamp'].apply(exchange.iso8601))

                    i+=1
                except (Exception,ccxt.ExchangeError, ccxt.AuthenticationError, ccxt.ExchangeNotAvailable, ccxt.RequestTimeout, ccxt.BadSymbol) as error:

                    print(logging.error(traceback.format_exc()))
                    i+=1
                    print('Got an error', type(error).__name__, error.args, ', retrying in', hold, 'seconds...')
                    time.sleep(hold)
                else:
                    data_pairs.at[i-1,'pair']=df
        data_pairs.to_pickle("data_pairs_binance_future_%s.pkl" % (timeframe))

        return data_pairs

    def coinex_margin(self,timeframe,limit):
        pairs = pd.read_pickle("pairs_coinex_margin.pkl")
        print('number of pairs in coinex_margin: ',len(pairs))
        save_pairs=pairs.copy()
        data_pairs=pairs.copy()
        i=0
        
        while i<len(pairs):
            try:

                symbol  =  save_pairs.at[i,'pair']
                tf =  {'1m':'1min','5m':'5min',
                               '15m':'15min','30m':'30min','1h':'1hour','2h':'2hour','4h':'4hour'
                              ,'6h':'6hour','12h':'12hour','1d':'1day','3d':'3day','1w':'1week'}
                duration =  tf[timeframe]

                headers = {
                        'Content-Type': 'application/json; charset=utf-8',
                        'Accept': 'application/json',}
                result = requests.get(
                    'https://api.coinex.com/v1/market/kline?market={market}&type={timeframe}&limit={limit}'.format(
                        market=symbol,timeframe=duration,limit=limit
                    ),
                    headers=headers
                )

                df = result.json()
                print('coinex_margin:',i)
                df=pd.DataFrame(df["data"])
                df.columns = ['Timestamp', 'open', 'close', 'high', 'low', 'volume', 'value']
                df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
                df['date'] = df['Timestamp'].dt.strftime("%d/%m/%Y")
                df['time'] = df['Timestamp'].dt.strftime("%H:%M:%S")
                df.drop(['value','Timestamp'], inplace=True, axis=1)
                df=df[["date", "time", "open", "high", "low", "close", "volume"]]
                i+=1
            except Exception as error:
                print(logging.error(traceback.format_exc()))
                i+=1
            else:
                data_pairs.at[i-1,'pair']=df
        data_pairs.to_pickle("data_pairs_coinex_margin_%s.pkl" % (timeframe))                
        return data_pairs

    def coinex_future(self,timeframe,limit):
        pairs = pd.read_pickle("pairs_coinex_future.pkl")
        print('number of pairs in coinex_future: ',len(pairs))
        save_pairs=pairs.copy()
        data_pairs=pairs.copy()
        i=0
        
        while i<len(pairs):
            try:

                symbol  =  save_pairs.at[i,'pair']
                tf =  {'1m':'1min','5m':'5min',
                               '15m':'15min','30m':'30min','1h':'1hour','2h':'2hour','4h':'4hour'
                              ,'6h':'6hour','12h':'12hour','1d':'1day','3d':'3day','1w':'1week'}
                duration =  tf[timeframe]                
                headers = {
                        'Content-Type': 'application/json; charset=utf-8',
                        'Accept': 'application/json',}
                result = requests.get(
                    'https://api.coinex.com/perpetual/v1/market/kline?market={market}&type={timeframe}&limit={limit}'.format(
                        market=symbol,timeframe=duration,limit=limit
                    ),
                    headers=headers
                )

                df = result.json()
                print('coinex_future:',i)
                df=pd.DataFrame(df["data"])
                df.columns = ['Timestamp', 'open', 'close', 'high', 'low', 'volume', 'value']
                df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
                df['date'] = df['Timestamp'].dt.strftime("%d/%m/%Y")
                df['time'] = df['Timestamp'].dt.strftime("%H:%M:%S")
                df.drop(['value','Timestamp'], inplace=True, axis=1)
                df=df[["date", "time", "open", "high", "low", "close", "volume"]]
                i+=1
            except Exception as error:
                print(logging.error(traceback.format_exc()))
                i+=1
            else:
                data_pairs.at[i-1,'pair']=df
        data_pairs.to_pickle("data_pairs_coinex_future_%s.pkl" % (timeframe))         
        return data_pairs

    def nobitex(self,timeframe,limit):
        pairs = pd.read_pickle("pairs_nobitex.pkl")
        print('number of pairs in nobitex: ',len(pairs))
        save_pairs=pairs.copy()
        data_pairs=pairs.copy()

        def number_candle(limit,timeframe):

            timeframes = (['1m',1,'minutes',limit*1],['3m',3,'minutes',limit*3],
                        ['5m',5,'minutes',limit*5],['15m',15,'minutes',limit*15],
                        ['30m',30,'minutes',limit*30],['1h',1,'hours',limit*1],
                        ['2h',2,'hours',limit*2],['4h',4,'hours',limit*4],
                        ['6h',6,'hours',limit*6],['8h',8,'hours',limit*8],
                        ['12h',12,'hours',limit*12],['1d',1,'days',limit*1],
                        ['3d',3,'days',limit*3],['1w',1,'weeks',limit*7])
            timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit'])
            timeframes.set_index('timeframe', inplace=True)

            timeframes['limit']  = timeframes['limit'].astype(np.float64)

            now = int(round(time.time() * 1000.0))
            if timeframes.loc[timeframe, 'name_time'] == 'minutes':
                from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

            if timeframes.loc[timeframe, 'name_time'] == 'hours':
                from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(hours=timeframes.loc[timeframe, 'limit'])

            if timeframes.loc[timeframe, 'name_time'] == 'days':
                from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(days=timeframes.loc[timeframe, 'limit'])



            from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S')
            fromdate = from_datetime
            return fromdate


        
        i=0
        while i<len(pairs):
            try:
                symbol     =  save_pairs.at[i,'pair']
                tf         =  {'1h':'60','3h':'180','6h':'360','12h':'720','1d':'D','2d':'2D','3d':'3D'}
                duration   =  tf[timeframe]

                fromdate   =  datetime.strptime(number_candle(limit,timeframe),'%Y-%m-%d %H:%M:%S').strftime("%d/%m/%Y %H:%M:%S")
                fromdate   =  int(time.mktime(datetime.strptime(str(fromdate), "%d/%m/%Y %H:%M:%S").timetuple()))

                todate     =  int((time.mktime((datetime.now()).timetuple())))
                url = "https://api.nobitex.ir/market/udf/history?symbol=%s&resolution=%s&from=%s&to=%s" % (symbol,duration,fromdate,todate)

                payload={}
                headers = {}

                response = requests.request("GET", url, headers=headers, data=payload)

                df=response.json()
                print('nobitex:',i)

                t=df['t']
                df=pd.DataFrame(df)
                df.columns = ['s','dateTime','close','open','high','low', 'volume']
                df['dateTime'] = pd.to_datetime(df.dateTime, unit='s')
                df['date'] = df.dateTime.dt.strftime("%d/%m/%Y")
                df['time'] = df.dateTime.dt.strftime("%H:%M:%S")
                df.drop(['s','dateTime'], inplace=True, axis=1)

                column_names = ["date", "time", "open", "high", "low", "close", "volume"]
                df = df.reindex(columns=column_names)
                df=df.astype({'open': 'int64', 'high': 'int64', 'low': 'int64', 'close': 'int64', 'volume': 'float'})
                i+=1
            except Exception as error:
                print(logging.error(traceback.format_exc()))
                i+=1
            else:
                data_pairs.at[i-1,'pair']=df
        data_pairs.to_pickle("data_pairs_nobitex_%s.pkl" % (timeframe))   
        return data_pairs

    def kucoin_margin(self,timeframe,limit):
        pairs = pd.read_pickle("pairs_kucoin_margin.pkl")
        print('number of pairs in kucoin_margin: ',len(pairs))
        save_pairs=pairs.copy()
        data_pairs=pairs.copy()



        def number_candle(timeframe,limit):

            timeframes = (['1m',1,'minutes',limit*1],['3m',3,'minutes',limit*3],
                        ['5m',5,'minutes',limit*5],['15m',15,'minutes',limit*15],
                        ['30m',30,'minutes',limit*30],['1h',1,'hours',limit*1],
                        ['2h',2,'hours',limit*2],['4h',4,'hours',limit*4],
                        ['6h',6,'hours',limit*6],['8h',8,'hours',limit*8],
                        ['12h',12,'hours',limit*12],['1d',1,'days',limit*1],
                        ['3d',3,'days',limit*3],['1w',1,'weeks',limit*7])
            timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit'])
            timeframes.set_index('timeframe', inplace=True)

            timeframes['limit']  = timeframes['limit'].astype(np.float64)

            now = int(round(time.time() * 1000.0))
            if timeframes.loc[timeframe, 'name_time'] == 'minutes':
                from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

            if timeframes.loc[timeframe, 'name_time'] == 'hours':
                from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(hours=timeframes.loc[timeframe, 'limit'])

            if timeframes.loc[timeframe, 'name_time'] == 'days':
                from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(days=timeframes.loc[timeframe, 'limit'])



            from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S')
            fromdate = from_datetime
            return fromdate

        i=0
        
        while i<len(pairs):
            try:
                symbol     =  save_pairs.at[i,'pair']
                tf         =  {'1m':'1min','3m':'3min','5m':'5min',
                               '15m':'15min','30m':'30min','1h':'1hour','2h':'2hour','4h':'4hour','6h':'6hour'
                              ,'8h':'8hour','12h':'12hour','1d':'1day','1w':'1week'}
                duration   =  tf[timeframe]
                fromdate   =  datetime.strptime(number_candle(timeframe,limit),'%Y-%m-%d %H:%M:%S').strftime("%d/%m/%Y %H:%M:%S")
                fromdate   =  int(time.mktime(datetime.strptime(str(fromdate), "%d/%m/%Y %H:%M:%S").timetuple()))

                todate     =  int((time.mktime((datetime.now()).timetuple())))

                url = "https://api.kucoin.com/api/v1/market/candles?type=%s&symbol=%s&startAt=%s&endAt=%s" % (duration,symbol,fromdate,todate)

                payload={}
                files={}
                headers ={}

                dff = requests.request("GET", url, headers=headers, data=payload, files=files)
                df=dff.json()
                print('kucoin_margin:',i)
                df=df['data']

                df=pd.DataFrame(df)
                df = df.rename({0:"Timestamp",1:"open",
                                2:"close",3:"high",4:"low",5:"volume",6:"value"}, axis='columns')
                df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='s')
                df['date'] = df['Timestamp'].dt.strftime("%d/%m/%Y")
                df['time'] = df['Timestamp'].dt.strftime("%H:%M:%S")
                df.drop(['value','Timestamp'], inplace=True, axis=1)
                df=df[["date", "time", "open", "high", "low", "close", "volume"]] 
                df =df[::-1]#reverse dataframe
                df.reset_index(drop=True,inplace=True)
                i+=1
            except Exception as error:
                print(logging.error(traceback.format_exc()))
                print(dff.json())
                i+=1
            else:
                data_pairs.at[i-1,'pair']=df
        data_pairs.to_pickle("data_pairs_kucoin_margin_%s.pkl" % (timeframe)) 
        return data_pairs

    def kucoin_future(self,timeframe,limit):
        pairs = pd.read_pickle("pairs_kucoin_future.pkl")
        print('number of pairs in kucoin_future: ',len(pairs))
        save_pairs=pairs.copy()
        data_pairs=pairs.copy()

        def number_candle(limit,timeframe):
            i=0
            list_date=[]
            n=limit/200
            while i<n:
                    timeframes = (['1m',1,'minutes',limit*1],['3m',3,'minutes',limit*3],
                                  ['5m',5,'minutes',limit*5],['15m',15,'minutes',limit*15],
                                  ['30m',30,'minutes',limit*30],['1h',1,'hours',limit*1],
                                  ['2h',2,'hours',limit*2],['4h',4,'hours',limit*4],
                                  ['6h',6,'hours',limit*6],['8h',8,'hours',limit*8],
                                  ['12h',12,'hours',limit*12],['1d',1,'days',limit*1],
                                  ['3d',3,'days',limit*3],['1w',1,'weeks',limit*7])
                    timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit'])
                    timeframes.set_index('timeframe', inplace=True)

                    timeframes['limit']  = timeframes['limit'].astype(np.float64)

                    now = int(round(time.time() * 1000.0))
                    if timeframes.loc[timeframe, 'name_time'] == 'minutes':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

                    if timeframes.loc[timeframe, 'name_time'] == 'hours':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(hours=timeframes.loc[timeframe, 'limit'])

                    if timeframes.loc[timeframe, 'name_time'] == 'days':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(days=timeframes.loc[timeframe, 'limit'])



                    from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S')
                    from_datetime=datetime.strptime(from_datetime,'%Y-%m-%d %H:%M:%S')
                    fromdate = from_datetime
                    fromdate =  int(from_datetime.timestamp() * 1000)
                    list_date.append(fromdate)
                    limit=limit-200
                    if limit<200:
                        todate =  int(round(time.time() * 1000.0))
                        list_date.append(todate)
                        break
                    i+=1
            return list_date

        j=0
        
        while j<=len(pairs):
            try:
                i=0
                df=[]
                list_date=number_candle(limit,timeframe)
                while i<(len(list_date)-1):

                        symbol  =  save_pairs.loc[j,'pair']
                        tf =  {'1m':1,'5m':5,
                                       '15m':15,'30m':30,'1h':60,'2h':120,'4h':240
                                      ,'8h':480,'12h':720,'1d':1440,'1w':10080}
                        duration =  tf[timeframe]
                        fromdate = list_date[i]

                        todate = list_date[i+1]

                        url = "https://api-futures.kucoin.com/api/v1/kline/query?symbol=%s&granularity=%s&from=%s&to=%s"\
                        % (symbol,duration,fromdate,todate)

                        payload={}
                        files={}
                        headers ={}

                        dfff = requests.request("GET", url, headers=headers, data=payload, files=files)
                        dff=dfff.json()
                        print('kucoin_future:',i)
                        dff=dff['data']
                        df.extend(dff)
                        i+=1
                df=pd.DataFrame(df)
                df = df.rename({0:"Timestamp",1:"open",
                                2:"high",3:"low",4:"close",5:"volume"}, axis='columns')
                df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='ms')
                df['date'] = df["Timestamp"].dt.strftime("%d/%m/%Y")
                df['time'] = df["Timestamp"].dt.strftime("%H:%M:%S")
                df.drop(['Timestamp'], inplace=True, axis=1)
                df=df[["date", "time", "open", "high", "low", "close", "volume"]] 

                j+=1
            except Exception as error:
                print(dfff.json(),'  in  ',i)
                print(logging.error(traceback.format_exc()))
                j+=1
            else:
                data_pairs.at[j-1,'pair']=df
        data_pairs.to_pickle("data_pairs_kucoin_future_%s.pkl" % (timeframe))
        return data_pairs

    def get_ohlcvs(self,limit):
        list_timeframe=['1h']
        i=0
        while i<len(list_timeframe):
#             if __name__ == '__main__':
            Thread(target = self.binance_margin,args=(list_timeframe[i],limit,)).start()
            Thread(target = self.binance_future,args=(list_timeframe[i],limit,)).start()
            Thread(target = self.coinex_margin,args=(list_timeframe[i],limit,)).start()
            Thread(target = self.coinex_future,args=(list_timeframe[i],limit,)).start()
            Thread(target = self.nobitex,args=(list_timeframe[i],limit,)).start()
            Thread(target = self.kucoin_margin,args=(list_timeframe[i],limit,)).start()
            Thread(target = self.kucoin_future,args=(list_timeframe[i],limit,)).start()
            i+=1


#اگر لیست تایم فریمها رو داخل نخ ها بزاریم قاطی میکنه چون هم زمان همه رو استفاده میکنه
#باید یک تابع مالتی پروسس ایجاد کنیم که داخلش حلقه وایل داشته باشه و  فقط چند پردازنده را استفاده کنیم و روی هر تایم 
#فریم که بود قفل بشه روی هر پروسس

#list_timeframe=['1m','5m','15m','30m','1h','2h','4h','6h','8h','12h','1d','1w']
d=ohlcvs()
d.get_ohlcvs(100)
#d.binance_margin('1h',100)


In [5]:
from datetime import timedelta, date, datetime
import time
import pandas as pd
import numpy as np
import ccxt
import requests
import traceback
import logging
import threading
from threading import Thread

def kucoin_future(timeframe,limit):
        pairs = pd.read_pickle("pairs_kucoin_future.pkl")
        print('number of pairs in kucoin_future: ',len(pairs))
        save_pairs=pairs.copy()
        data_pairs=pairs.copy()

        def number_candle(limit,timeframe):
            i=0
            list_date=[]
            n=limit/200
            while i<n:
                    timeframes = (['1m',1,'minutes',limit*1],['3m',3,'minutes',limit*3],
                                  ['5m',5,'minutes',limit*5],['15m',15,'minutes',limit*15],
                                  ['30m',30,'minutes',limit*30],['1h',1,'hours',limit*1],
                                  ['2h',2,'hours',limit*2],['4h',4,'hours',limit*4],
                                  ['6h',6,'hours',limit*6],['8h',8,'hours',limit*8],
                                  ['12h',12,'hours',limit*12],['1d',1,'days',limit*1],
                                  ['3d',3,'days',limit*3],['1w',1,'weeks',limit*7])
                    timeframes = pd.DataFrame(timeframes,columns=['timeframe' , 'time' , 'name_time' , 'limit'])
                    timeframes.set_index('timeframe', inplace=True)

                    timeframes['limit']  = timeframes['limit'].astype(np.float64)

                    now = int(round(time.time() * 1000.0))
                    if timeframes.loc[timeframe, 'name_time'] == 'minutes':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(minutes=timeframes.loc[timeframe, 'limit'])

                    if timeframes.loc[timeframe, 'name_time'] == 'hours':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(hours=timeframes.loc[timeframe, 'limit'])

                    if timeframes.loc[timeframe, 'name_time'] == 'days':
                        from_datetime=datetime.fromtimestamp(now/1000.0) - timedelta(days=timeframes.loc[timeframe, 'limit'])



                    from_datetime=datetime.strftime(from_datetime, '%Y-%m-%d %H:%M:%S')
                    from_datetime=datetime.strptime(from_datetime,'%Y-%m-%d %H:%M:%S')
                    fromdate = from_datetime
                    fromdate =  int(from_datetime.timestamp() * 1000)
                    list_date.append(fromdate)
                    limit=limit-200
                    if limit<200:
                        todate =  int(round(time.time() * 1000.0))
                        list_date.append(todate)
                        break
                    i+=1
            return list_date

        j=0
        list_date=number_candle(limit,timeframe)
        print(list_date)
        while j<=len(pairs):
            try:
                i=0
                df=[]
                list_date=number_candle(limit,timeframe)
                while i<(len(list_date)-1):

                        symbol  =  save_pairs.loc[j,'pair']
                        tf =  {'1m':1,'5m':5,
                                       '15m':15,'30m':30,'1h':60,'2h':120,'4h':240
                                      ,'8h':480,'12h':720,'1d':1440,'1w':10080}
                        duration =  tf[timeframe]
                        fromdate = list_date[i]

                        todate = list_date[i+1]

                        url = "https://api-futures.kucoin.com/api/v1/kline/query?symbol=%s&granularity=%s&from=%s&to=%s"\
                        % (symbol,duration,fromdate,todate)

                        payload={}
                        files={}
                        headers ={}

                        dfff = requests.request("GET", url, headers=headers, data=payload, files=files)
                        dff=dfff.json()
                        
                        dff=dff['data']
                        df.extend(dff)
                        i+=1
                print('kucoin_future:',j)
                df=pd.DataFrame(df)
                df = df.rename({0:"Timestamp",1:"open",
                                2:"high",3:"low",4:"close",5:"volume"}, axis='columns')
                df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='ms')
                df['date'] = df["Timestamp"].dt.strftime("%d/%m/%Y")
                df['time'] = df["Timestamp"].dt.strftime("%H:%M:%S")
                df.drop(['Timestamp'], inplace=True, axis=1)
                df=df[["date", "time", "open", "high", "low", "close", "volume"]] 

                j+=1
            except Exception as error:
                print(dfff.json(),'  in  ',i)
                print(logging.error(traceback.format_exc()))
                j+=1
            else:
                data_pairs.at[j-1,'pair']=df
        data_pairs.to_pickle("data_pairs_kucoin_future_%s.pkl" % (timeframe))
        return data_pairs
kucoin_future('1h',400)

number of pairs in kucoin_future:  103
[1650400560000, 1651120560000, 1651840560873]
kucoin_future: 0
kucoin_future: 1
kucoin_future: 2
kucoin_future: 3
kucoin_future: 4


KeyboardInterrupt: 